In [1]:
import pandas as pd
import re
import sys
import requests
import pandas as pd
import sqlite3
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
conn = sqlite3.connect("../test.sqlite3")
import numpy as np

# ---------------- Pandas settings --------------- #
# Removes rows and columns truncation of '...'
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

## Reading Trials data

In [2]:
trials = pd.read_csv("../data/cleaned_trial_df.csv")
trials = trials.drop(columns=["Unnamed: 0"], axis=1)
trials.head()

,trialid,registry,registration_date,enrollment_date,sponsors,recruitment_status,phase,study_type,countries,title,intervention_type,intervention,data_source,results_link,last_refreshed_on,first_seen
0,chictr2000029949,chictr,2020-02-16,2020-02-16,zhongnan hospital of wuhan university,not recruiting,not applicable,observational,china,a medical records based study for the effectiv...,drug,oxygen,http://www.chictr.org.cn/showproj.aspx?proj=49181,no results,2020-02-17,2020-08-03
1,chictr2000029953,chictr,2020-02-17,2020-02-01,zhongnan hospital of wuhan university,not recruiting,not applicable,observational,china,construction and analysis of prognostic predic...,prognosis,no intervention,http://www.chictr.org.cn/showproj.aspx?proj=49217,no results,2020-02-17,2020-08-03
2,chictr2000029935,chictr,2020-02-16,2020-02-06,"hwa mei hospital,university of chinese academy...",recruiting,not applicable,interventional,china,a single-arm clinical trial for chloroquine ph...,drug,chloroquine,http://www.chictr.org.cn/showproj.aspx?proj=49607,no results,2020-02-17,2020-08-03
3,chictr2000029947,chictr,2020-02-16,2020-03-01,longhua hospital affiliated to shanghai univer...,not recruiting,not applicable,interventional,china,a randomized controlled trial for qingyi no. 4...,traditional medicine (drug),traditional medicine,http://www.chictr.org.cn/showproj.aspx?proj=49599,no results,2020-02-17,2020-03-18
4,chictr2000029896,chictr,2020-02-16,2020-01-31,affiliated hospital of changchun university of...,recruiting,not applicable,observational,china,evaluate the effectiveness of traditional chin...,traditional medicine (unspecified),traditional medicine,http://www.chictr.org.cn/showproj.aspx?proj=49432,no results,2020-02-17,2020-03-18


In [3]:
trials["intervention"].value_counts().index.to_list()

['no intervention',
 'traditional medicine',
 'cells,mesenchymal',
 'plasma',
 'psychological',
 'hydroxychloroquine',
 'chloroquine',
 'favipiravir',
 'corticosteroids',
 'remdesivir',
 'oxygen',
 'vaccine',
 'exercise',
 'interferon beta',
 'tocilizumab',
 'nitric oxide',
 'pirfenidone',
 'azvudine',
 'interferon alfa',
 'physio',
 'gene-derived protein',
 'renal replacement therapy',
 'cells,stem cells',
 'probiotics',
 'mechanical ventilation',
 'umifenovir',
 'immunoglobulin',
 'lopinavir-ritonavir',
 'ozonated autohemotherapy',
 'danoprevir/ritonavir',
 'combination: hydroxychloroquine vs chloroquine',
 'bevacizumab',
 'endotracheal intubation',
 'intestinal microbiome',
 'carrimycin',
 'adenovirus vector',
 'nutrition',
 'lmwh (enoxaparin)',
 'humanistic care',
 'pul-042,unspecified immunomodulatory drug',
 'cells,mononuclear',
 'arb (losartan)',
 'interferon',
 'combination: pd1 + thymosin',
 'interleukkn-6 monoclonal antibody (cmab806)',
 'combination: favipiravir + tocilizuma

## Reading Products

In [4]:
products = pd.read_sql_query("SELECT * FROM `productraw`;", conn)
products.head()

,product_id,preferred_name,chemical_name,brand_name,repurposed,notes,disease,application,data_reference,data_source,product_type,sponsors,intervention_type,indication,molecule_type,therapeutic_approach,other_partners,num_sites,site_locations
0,1,mrna-1273,mrna-1273,None,new,None,None,None,None,None,None,moderna; national institute of allergy and inf...,vaccine - prophylactic,covid-19,nucleic acid based therapies/vaccines,unknown,cepi,2.0,kaiser permanente washington health research i...
1,2,novavax candidate,novavax candidate,None,None,None,None,None,None,None,None,novavax inc.; emergent biosolutions inc.,vaccine - prophylactic,covid-19,None,None,None,NaN,None
2,3,bnt162,bnt162,None,None,None,None,None,None,None,None,pfizer inc.; biontech se,vaccine - prophylactic,covid-19,None,None,None,NaN,None
3,4,ino-4800,ino-4800,None,None,None,None,None,None,None,None,inovio pharmaceuticals inc.; beijing advaccine...,vaccine - prophylactic,covid-19,None,None,None,NaN,None
4,5,regn-eb3,None,regn-eb3,None,None,None,None,None,None,None,regeneron; hhs,therapeutic,covid-19,None,None,None,NaN,None


## Create Inferred product column

### Create products list

In [5]:
products_names = products["preferred_name"].str.replace(" ", "").str.lower().to_list()
products_names

['mrna-1273',
 'novavaxcandidate',
 'bnt162',
 'ino-4800',
 'regn-eb3',
 'chadox1',
 'delns1-sars-cov2-rbdlaiv',
 'ad5-ncov',
 'ino-4700',
 'aviancoronavirusinfectiousbronchitisvirus(ibv)vaccine',
 'tnx-1800',
 'recombinantsubunitvaccine',
 'lineardnavaccine',
 'remdesivir',
 'lopinavir',
 'sarilumab',
 'hydroxycholoroquine',
 'favilavir',
 'tjm2',
 'at-100',
 'tzls-501',
 'oya1',
 'bpi-002',
 'np-120(ifenprodil)',
 'apn01',
 'brilacidin',
 'leronlimab',
 'bxt-25',
 'actemra',
 'galidesivir',
 'regn3048-3051',
 'sng001',
 'amnioboost',
 'favipiravir',
 'mrna-1273',
 'novavaxcandidate',
 'bnt162',
 'ino-4800',
 'regn-eb3',
 'chadox1',
 'delns1-sars-cov2-rbdlaiv',
 'ad5-ncov',
 'ino-4700',
 'aviancoronavirusinfectiousbronchitisvirus(ibv)vaccine',
 'tnx-1800',
 'recombinantsubunitvaccine',
 'lineardnavaccine',
 'remdesivir',
 'lopinavir',
 'sarilumab',
 'hydroxycholoroquine',
 'favilavir',
 'tjm2',
 'at-100',
 'tzls-501',
 'oya1',
 'bpi-002',
 'np-120(ifenprodil)',
 'apn01',
 'brilacidin'

In [6]:
trials[trials["title"].str.contains(products_names[0])]

,trialid,registry,registration_date,enrollment_date,sponsors,recruitment_status,phase,study_type,countries,title,intervention_type,intervention,data_source,results_link,last_refreshed_on,first_seen
490,nct04283461,clinicaltrials.gov,2020-02-21,2020-03-03,national institute of allergy and infectious d...,recruiting,phase 1,interventional,united states,safety and immunogenicity study of 2019-ncov v...,atmp,vaccine,https://clinicaltrials.gov/show/nct04283461,no results,2020-03-23,2020-08-03


In [7]:
df = trials.copy()

In [8]:
# df["product_name"] = np.NaN
# for i in range(len(products_names)):
#     if (df["title"].str.contains(products_names[i])) & (df["product_name"].isna()):
#         df["product_name"] = products_names[i]
        
# df["product_name"].value_counts(dropna=False)

In [9]:
df["title"].loc[2]

'a single-arm clinical trial for chloroquine phosphate in the treatment of novel coronavirus pneumonia 2019 (covid-19)'

In [10]:
def get_name(val):
    list_vals = val.split()#.replace(string.punctuations, "")
    matches = []
    for name in products_names:
        if fuzz.partial_ratio(name, list_vals) > 80:
            matches.append(name)
    return ",".join(matches)

df["product_name"] = df["title"].apply(get_name)
df["product_name"].value_counts()

                                                                                             586
hydroxycholoroquine,hydroxycholoroquine,hydroxycholoroquine                                   12
favipiravir,favipiravir,favipiravir                                                           10
lopinavir,lopinavir,lopinavir                                                                  9
remdesivir,remdesivir,remdesivir                                                               5
sarilumab,sarilumab,sarilumab                                                                  1
mrna-1273,mrna-1273,mrna-1273                                                                  1
lopinavir,favipiravir,lopinavir,favipiravir,lopinavir,favipiravir                              1
lopinavir,hydroxycholoroquine,lopinavir,hydroxycholoroquine,lopinavir,hydroxycholoroquine      1
Name: product_name, dtype: int64

In [11]:
df["search_string"] = df["title"] + " " + df["intervention"]
df.head()

,trialid,registry,registration_date,enrollment_date,sponsors,recruitment_status,phase,study_type,countries,title,intervention_type,intervention,data_source,results_link,last_refreshed_on,first_seen,product_name,search_string
0,chictr2000029949,chictr,2020-02-16,2020-02-16,zhongnan hospital of wuhan university,not recruiting,not applicable,observational,china,a medical records based study for the effectiv...,drug,oxygen,http://www.chictr.org.cn/showproj.aspx?proj=49181,no results,2020-02-17,2020-08-03,,a medical records based study for the effectiv...
1,chictr2000029953,chictr,2020-02-17,2020-02-01,zhongnan hospital of wuhan university,not recruiting,not applicable,observational,china,construction and analysis of prognostic predic...,prognosis,no intervention,http://www.chictr.org.cn/showproj.aspx?proj=49217,no results,2020-02-17,2020-08-03,,construction and analysis of prognostic predic...
2,chictr2000029935,chictr,2020-02-16,2020-02-06,"hwa mei hospital,university of chinese academy...",recruiting,not applicable,interventional,china,a single-arm clinical trial for chloroquine ph...,drug,chloroquine,http://www.chictr.org.cn/showproj.aspx?proj=49607,no results,2020-02-17,2020-08-03,,a single-arm clinical trial for chloroquine ph...
3,chictr2000029947,chictr,2020-02-16,2020-03-01,longhua hospital affiliated to shanghai univer...,not recruiting,not applicable,interventional,china,a randomized controlled trial for qingyi no. 4...,traditional medicine (drug),traditional medicine,http://www.chictr.org.cn/showproj.aspx?proj=49599,no results,2020-02-17,2020-03-18,,a randomized controlled trial for qingyi no. 4...
4,chictr2000029896,chictr,2020-02-16,2020-01-31,affiliated hospital of changchun university of...,recruiting,not applicable,observational,china,evaluate the effectiveness of traditional chin...,traditional medicine (unspecified),traditional medicine,http://www.chictr.org.cn/showproj.aspx?proj=49432,no results,2020-02-17,2020-03-18,,evaluate the effectiveness of traditional chin...


In [12]:
df["product_name"] = df["search_string"].apply(get_name)
df["product_name"].value_counts()

                                                                                                   579
hydroxycholoroquine,hydroxycholoroquine,hydroxycholoroquine                                         14
favipiravir,favipiravir,favipiravir                                                                 10
lopinavir,lopinavir,lopinavir                                                                        9
remdesivir,remdesivir,remdesivir                                                                     6
lopinavir,hydroxycholoroquine,lopinavir,hydroxycholoroquine,lopinavir,hydroxycholoroquine            3
remdesivir,hydroxycholoroquine,remdesivir,hydroxycholoroquine,remdesivir,hydroxycholoroquine         1
sarilumab,sarilumab,sarilumab                                                                        1
hydroxycholoroquine,favipiravir,hydroxycholoroquine,favipiravir,hydroxycholoroquine,favipiravir      1
mrna-1273,mrna-1273,mrna-1273                                            

In [13]:
df.loc[df["product_name"] == '', "product_name"] = None
df["product_name"].value_counts(dropna=False)

NaN                                                                                                579
hydroxycholoroquine,hydroxycholoroquine,hydroxycholoroquine                                         14
favipiravir,favipiravir,favipiravir                                                                 10
lopinavir,lopinavir,lopinavir                                                                        9
remdesivir,remdesivir,remdesivir                                                                     6
lopinavir,hydroxycholoroquine,lopinavir,hydroxycholoroquine,lopinavir,hydroxycholoroquine            3
remdesivir,hydroxycholoroquine,remdesivir,hydroxycholoroquine,remdesivir,hydroxycholoroquine         1
sarilumab,sarilumab,sarilumab                                                                        1
hydroxycholoroquine,favipiravir,hydroxycholoroquine,favipiravir,hydroxycholoroquine,favipiravir      1
mrna-1273,mrna-1273,mrna-1273                                            

In [14]:
df.shape

(626, 18)

# 1. Vaccines

In [15]:
vaccines = df[df["intervention"].str.contains("vaccine")]
vaccines.head()

,trialid,registry,registration_date,enrollment_date,sponsors,recruitment_status,phase,study_type,countries,title,intervention_type,intervention,data_source,results_link,last_refreshed_on,first_seen,product_name,search_string
28,chictr2000029636,chictr,2020-02-08,2020-02-09,"union hospital,tongji medical college,huazhong...",recruiting,not applicable,interventional,china,efficacy and safety of aerosol inhalation of v...,atmp,vaccine,http://www.chictr.org.cn/showproj.aspx?proj=49215,no results,2020-02-17,2020-08-03,None,efficacy and safety of aerosol inhalation of v...
132,chictr2000030016,chictr,2020-02-19,2020-02-04,guangxi medical university,recruiting,phase 4,interventional,china,basic and clinical study of inhalation of inac...,atmp,vaccine,http://www.chictr.org.cn/showproj.aspx?proj=49799,no results,2020-02-25,2020-08-03,None,basic and clinical study of inhalation of inac...
444,chictr2000030750,chictr,2020-03-13,2020-03-01,shenzhen third people's hospital,not recruiting,phase 1/phase 2,interventional,china,a clinical study for effectiveness and safety ...,atmp,vaccine,http://www.chictr.org.cn/showproj.aspx?proj=50928,no results,2020-03-16,2020-03-18,None,a clinical study for effectiveness and safety ...
488,nct04276896,clinicaltrials.gov,2020-02-17,2020-03-24,shenzhen geno-immune medical institute,recruiting,phase 1/phase 2,interventional,china,immunity and safety of covid-19 synthetic mini...,atmp,vaccine,https://clinicaltrials.gov/show/nct04276896,no results,2020-03-23,2020-03-18,None,immunity and safety of covid-19 synthetic mini...
490,nct04283461,clinicaltrials.gov,2020-02-21,2020-03-03,national institute of allergy and infectious d...,recruiting,phase 1,interventional,united states,safety and immunogenicity study of 2019-ncov v...,atmp,vaccine,https://clinicaltrials.gov/show/nct04283461,no results,2020-03-23,2020-08-03,"mrna-1273,mrna-1273,mrna-1273",safety and immunogenicity study of 2019-ncov v...


In [16]:
vaccines.shape

(5, 18)

# 2. Treatments

In [17]:
treatments = df[(df["intervention_type"].str.contains("traditional") | df["intervention_type"].str.contains("drug"))]
treatments.head()

,trialid,registry,registration_date,enrollment_date,sponsors,recruitment_status,phase,study_type,countries,title,intervention_type,intervention,data_source,results_link,last_refreshed_on,first_seen,product_name,search_string
0,chictr2000029949,chictr,2020-02-16,2020-02-16,zhongnan hospital of wuhan university,not recruiting,not applicable,observational,china,a medical records based study for the effectiv...,drug,oxygen,http://www.chictr.org.cn/showproj.aspx?proj=49181,no results,2020-02-17,2020-08-03,None,a medical records based study for the effectiv...
2,chictr2000029935,chictr,2020-02-16,2020-02-06,"hwa mei hospital,university of chinese academy...",recruiting,not applicable,interventional,china,a single-arm clinical trial for chloroquine ph...,drug,chloroquine,http://www.chictr.org.cn/showproj.aspx?proj=49607,no results,2020-02-17,2020-08-03,None,a single-arm clinical trial for chloroquine ph...
3,chictr2000029947,chictr,2020-02-16,2020-03-01,longhua hospital affiliated to shanghai univer...,not recruiting,not applicable,interventional,china,a randomized controlled trial for qingyi no. 4...,traditional medicine (drug),traditional medicine,http://www.chictr.org.cn/showproj.aspx?proj=49599,no results,2020-02-17,2020-03-18,None,a randomized controlled trial for qingyi no. 4...
4,chictr2000029896,chictr,2020-02-16,2020-01-31,affiliated hospital of changchun university of...,recruiting,not applicable,observational,china,evaluate the effectiveness of traditional chin...,traditional medicine (unspecified),traditional medicine,http://www.chictr.org.cn/showproj.aspx?proj=49432,no results,2020-02-17,2020-03-18,None,evaluate the effectiveness of traditional chin...
5,chictr2000029811,chictr,2020-02-14,2020-02-20,guangzhou reborn health management consultatio...,not recruiting,not applicable,interventional,no country given,clinical study for anti-aging active freeze-dr...,traditional medicine (drug),traditional medicine,http://www.chictr.org.cn/showproj.aspx?proj=49355,no results,2020-02-17,2020-08-03,None,clinical study for anti-aging active freeze-dr...


In [18]:
treatments["intervention_type"].value_counts()

drug                                       158
traditional medicine (drug)                 59
traditional medicine (unspecified)          35
traditional medicine (other)                24
traditional medicine + western medicine     23
other (drug)                                 1
drug (+atmp +other (renal))                  1
drug (+atmp)                                 1
Name: intervention_type, dtype: int64

In [19]:
treatments.shape

(302, 18)

## 3. Alternative Stuff

In [20]:
alternatives = df[~(df["intervention_type"].str.contains("traditional") | df["intervention_type"].str.contains("drug")) & ~(df["intervention"].str.contains("vaccine"))]
alternatives.head()

,trialid,registry,registration_date,enrollment_date,sponsors,recruitment_status,phase,study_type,countries,title,intervention_type,intervention,data_source,results_link,last_refreshed_on,first_seen,product_name,search_string
1,chictr2000029953,chictr,2020-02-17,2020-02-01,zhongnan hospital of wuhan university,not recruiting,not applicable,observational,china,construction and analysis of prognostic predic...,prognosis,no intervention,http://www.chictr.org.cn/showproj.aspx?proj=49217,no results,2020-02-17,2020-08-03,None,construction and analysis of prognostic predic...
6,chictr2000029810,chictr,2020-02-14,2020-02-16,shenzhen second people's hospital,recruiting,not applicable,diagnostic test,china,clinical study of a novel high sensitivity nuc...,diagnosis,no intervention,http://www.chictr.org.cn/showproj.aspx?proj=49407,no results,2020-02-17,2020-08-03,None,clinical study of a novel high sensitivity nuc...
7,chictr2000029850,chictr,2020-02-15,2020-02-15,the first affiliated hospital of zhejiang univ...,recruiting,not applicable,interventional,china,study on convalescent plasma treatment for sev...,atmp,plasma,http://www.chictr.org.cn/showproj.aspx?proj=49533,no results,2020-02-17,2020-08-03,None,study on convalescent plasma treatment for sev...
9,chictr2000029779,chictr,2020-02-13,2020-02-14,wuxi people's hospital,recruiting,not applicable,diagnostic test,china,study for the key issues of the diagnosis and ...,diagnosis,no intervention,http://www.chictr.org.cn/showproj.aspx?proj=49378,no results,2020-02-17,2020-08-03,None,study for the key issues of the diagnosis and ...
11,chictr2000029805,chictr,2020-02-14,2020-02-12,wuhan jinyintan hospital (wuhan infectious dis...,recruiting,not applicable,observational,china,analysis of clinical characteristics of severe...,clinical presentation,no intervention,http://www.chictr.org.cn/showproj.aspx?proj=49188,no results,2020-02-17,2020-08-03,None,analysis of clinical characteristics of severe...


In [22]:
alternatives["intervention_type"].value_counts()

clinical presentation    124
atmp                      49
diagnosis                 43
prognosis                 33
other (psych)             19
other (physical)          12
other                     11
genomics                   7
other (physio)             4
supplement                 4
other (exercise)           4
other (renal)              4
other (nutrition)          2
other (device)             1
other (radiology)          1
oher (psych)               1
Name: intervention_type, dtype: int64

In [23]:
alternatives.shape

(319, 18)

## Fix country route

In [25]:
trials["countries"].value_counts()

china                                                   526
no country given                                         52
france                                                    8
iran                                                      8
united states                                             6
japan                                                     4
italy                                                     4
united kingdom                                            3
hong kong                                                 2
hong kong,united states,south korea,taiwan,singapore      2
jordan                                                    1
south korea,singapore,united states                       1
israel,united states                                      1
japan,australia,asia,europe                               1
thailand                                                  1
south korea                                               1
singapore                               